# Classification Toy problems

### Imports and utils

#### Imports

In [68]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

from datetime import datetime
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
import torch
from torch import tensor, Tensor
from torch.nn import Sequential, Linear, BCELoss, Sigmoid, Tanh, Module, CrossEntropyLoss, ReLU, Softmax, NLLLoss, KLDivLoss, L1Loss, MSELoss
import torch.nn as nn
from torch.optim import Adam, SGD, Adagrad, Adadelta, AdamW, RMSprop, Optimizer

In [ ]:
class History:
    def __init__(self) -> None:
        self.history = {}
        self.epoch = []

def model_forward_func(model: Module):
    return lambda x: model(torch.tensor(x, dtype=torch.float32)).detach().squeeze().numpy()
        
def to_categorical(x, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.

    E.g. for use with `categorical_crossentropy`.

    Args:
        x: Array-like with class values to be converted into a matrix
            (integers from 0 to `num_classes - 1`).
        num_classes: Total number of classes. If `None`, this would be inferred
            as `max(x) + 1`. Defaults to `None`.

    Returns:
        A binary matrix representation of the input as a NumPy array. The class
        axis is placed last.

    Example:

    >>> a = keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
    >>> print(a)
    [[1. 0. 0. 0.]
     [0. 1. 0. 0.]
     [0. 0. 1. 0.]
     [0. 0. 0. 1.]]

    >>> b = np.array([.9, .04, .03, .03,
    ...               .3, .45, .15, .13,
    ...               .04, .01, .94, .05,
    ...               .12, .21, .5, .17],
    ...               shape=[4, 4])
    >>> loss = keras.ops.categorical_crossentropy(a, b)
    >>> print(np.around(loss, 5))
    [0.10536 0.82807 0.1011  1.77196]

    >>> loss = keras.ops.categorical_crossentropy(a, a)
    >>> print(np.around(loss, 5))
    [0. 0. 0. 0.]
    """
    x = np.array(x, dtype="int64")
    input_shape = x.shape

    # Shrink the last dimension if the shape is (..., 1).
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])

    x = x.reshape(-1)
    if not num_classes:
        num_classes = np.max(x) + 1
    batch_size = x.shape[0]
    categorical = np.zeros((batch_size, num_classes))
    categorical[np.arange(batch_size), x] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

#### Plotting Utils

In [ ]:
from matplotlib.axes import Axes

def plot_decision_boundary(func, X, y, figsize=(7, 5), ax: Axes = None):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    c = func(ab)
    cc = c.reshape(aa.shape)

    cm = plt.colormaps['RdBu']
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111)

    contour = ax.contourf(aa, bb, cc, cmap=cm, alpha=0.8)
    
    ax_c = plt.colorbar(contour, ax=ax)
    ax_c.set_label("$P(y = 1)$")
    ax_c.set_ticks([0, 0.25, 0.5, 0.75, 1])
    
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    ax.set_xlim(amin, amax)
    ax.set_ylim(bmin, bmax)
    ax.set_title("Decision Boundary")
    return ax

def plot_multiclass_decision_boundary(model: Module, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    cmap = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    Z = model.forward(tensor(np.c_[xx.ravel(), yy.ravel()])).detach().numpy()
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1,1,1)
    ax.contourf(xx, yy, Z, cmap=plt.colormaps['Spectral'], alpha=0.8)
    ax.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.colormaps['RdYlBu'])
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    return ax
    
def plot_data(X, y, figsize=(6, 4)):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1,1,1)
    ax.plot(X[y==0, 0], X[y==0, 1], 'or', alpha=0.5, label=0)
    ax.plot(X[y==1, 0], X[y==1, 1], 'ob', alpha=0.5, label=1)
    ax.set_xlim((min(X[:, 0])-0.1, max(X[:, 0])+0.1))
    ax.set_ylim((min(X[:, 1])-0.1, max(X[:, 1])+0.1))
    ax.legend()
    return ax

def plot_loss_accuracy(history: History, ax: Axes = None):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    if ax is None:
        fig = plt.figure(figsize=(6, 4))
        ax = fig.add_subplot(111)
    historydf.plot(ylim=(0, max(1, historydf.values.max())), ax=ax)
    loss = history.history['loss'][-1]
    acc = history.history['acc'][-1]
    ax.set_title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))
    return  ax

def plot_loss(history: History):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(111)
    historydf.plot(ylim=(0, historydf.values.max()), ax=ax)
    fig.suptitle('Loss: %.3f' % history.history['loss'][-1])
    return fig, ax

def plot_confusion_matrix(y_pred, y, ax: Axes = None):
    if ax is None:
        fig = plt.figure(figsize=(6, 4))
        ax = fig.add_subplot(1,1,1)

    sns.heatmap(pd.DataFrame(confusion_matrix(y, y_pred)), annot=True, fmt='d', cmap='YlGnBu', alpha=0.8, vmin=0, ax=ax)
    ax.set_title("Confusion Matrix")
    return ax

def plot_compare_histories(history_list: list[History], name_list, plot_accuracy=True):
    dflist = []
    for history in history_list:
        h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
        dflist.append(pd.DataFrame(h, index=history.epoch))

    historydf = pd.concat(dflist, axis=1)

    metrics = dflist[0].columns
    idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
    historydf.columns = idx
    
    fig = plt.figure(figsize=(6, 8))

    ax = fig.add_subplot(211)
    historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
    ax.set_title("Loss")
    
    if plot_accuracy:
        ax = fig.add_subplot(212)
        historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
        ax.set_title("Accuracy")
        ax.set_xlabel("Epochs")

    plt.tight_layout()
    
def make_sine_wave():
    c = 3
    num = 2400
    step = num/(c*4)
    np.random.seed(0)
    x0 = np.linspace(-c*np.pi, c*np.pi, num)
    x1 = np.sin(x0)
    noise = np.random.normal(0, 0.1, num) + 0.1
    noise = np.sign(x1) * np.abs(noise)
    x1  = x1 + noise
    x0 = x0 + (np.asarray(range(num)) / step) * 0.3
    X = np.column_stack((x0, x1))
    y = np.asarray([int((i/step)%2==1) for i in range(len(x0))])
    return X, y

def make_multiclass(N=500, D=2, K=3):
    """
    N: number of points per class
    D: dimensionality
    K: number of classes
    """
    np.random.seed(0)
    X = np.zeros((N*K, D))
    y = np.zeros(N*K)
    for j in range(K):
        ix = range(N*j, N*(j+1))
        # radius
        r = np.linspace(0.0,1,N)
        # theta
        t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)
    ax.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.colormaps['RdYlBu'], alpha=0.8)
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    return X, y

#### Simple Classification dataset

In [ ]:
def get_classification_data(plot=False):
    X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                           n_informative=2, random_state=7, n_clusters_per_class=1)
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)
    if plot:
        plot_data(X, y)
    return X, y, X_tensor, y_tensor


In [ ]:
# X, y, X_ten, y_ten = get_classification_data()

### Simple Sklearn Logistic Regression

In [ ]:
def simple_sklearn_logistic_regression(X, y):
    lr = LogisticRegression()
    lr.fit(X, y)
    print('LR coefficients:', lr.coef_)
    print('LR intercept:', lr.intercept_)

    plot_data(X, y)

    limits = np.array([-2, 2])
    boundary = -(lr.coef_[0][0] * limits + lr.intercept_[0]) / lr.coef_[0][1]
    plt.plot(limits, boundary, "g-", linewidth=2)

### Torch Logistic Regression

In [ ]:
def get_logistic_regression_model():
    model = Sequential(Linear(2, 1), Sigmoid())
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.1)
    return model, criterion, optimizer
# model = Sequential(Linear(2, 1), Sigmoid())
# criterion = BCELoss()
# optimizer = SGD(model.parameters(), lr=0.1)

#### basic trainer

In [ ]:
def trainer(model: Module, X_tensor: Tensor, y_tensor: Tensor, optimizer, criterion, epochs: int):
    history = History()
    history.history['loss'] = []
    history.history['acc'] = []
    # history.history['acc2'] = []

    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor.view(-1, 1))
        # print(y_tensor.shape)
        # print(y_pred.shape)

        # print(y_tensor.view(-1,1).shape)
        loss.backward()
        optimizer.step()
        y_pred_d = y_pred > 0.5
        acc = (y_pred_d == y_tensor.view(-1, 1)).sum().item() / len(y_tensor)

        # total = 0
        # correct = 0
        # total += y_tensor.size(0)
        # correct += torch.sum(y_pred.round().detach() == y_tensor.view(-1,1)).item()
        # accuracy = correct/total
        
        # history.history['acc2'].append(accuracy)
        history.history['loss'].append(loss.item())
        history.history['acc'].append(acc)
        history.epoch.append(epoch)
        if epoch % 10 == 0:
            print('Epoch:', epoch, 'Loss:', loss.item())
    return model, history

#### Log regr for simple classification data

In [ ]:
X, y, X_tensor, y_tensor = get_classification_data()
model, criterion, optimizer = get_logistic_regression_model()
model, history = trainer(model, X_tensor, y_tensor, optimizer, criterion, 50)
y_pred = (model(X_tensor) > 0.5).detach().numpy()

In [ ]:
master_fig, axs = plt.subplots(1,3 ,figsize=(16, 4), width_ratios=[6,8,6])
ax1 = plot_loss_accuracy(history, ax=axs[0])
ax2 = plot_decision_boundary(model_forward_func(model), X, y, ax=axs[1])
ax3 = plot_confusion_matrix(y_pred, y, ax=axs[2])
# print(classification_report(y, y_pred))


## Moons

In [ ]:
def get_moons_data(plot=False):
    X, y = make_moons(n_samples=1000, noise=0.05, random_state=0)
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)
    if plot:
        plot_data(X, y)
    return X, y, X_tensor, y_tensor

In [ ]:

model = Sequential(Linear(2, 1), Sigmoid())
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.1)
model, history = trainer(model, X_tensor, y_tensor, optimizer, criterion, 100)
plot_loss_accuracy(history)

In [ ]:
plot_decision_boundary(model_forward_func(model), X, y )

In [ ]:
y_pred = model(X_tensor).squeeze().round().detach().numpy()
print(classification_report(y, y_pred))
plot_confusion_matrix(y_pred, y)

### ANN

In [ ]:
model = Sequential(
    Linear(2,4),
    Tanh(),
    Linear(4,2),
    Tanh(),
    Linear(2,1),
    Sigmoid()
)
criterion = BCELoss()
optimizer = Adam(model.parameters(), lr=0.1)
model, history = trainer(model, X_tensor, y_tensor, optimizer, criterion, 100)
plot_loss_accuracy(history)

In [ ]:
plot_decision_boundary(model_forward_func(model), X, y)

In [ ]:
X, y = make_circles(n_samples=1000, noise=0.05, factor=0.3, random_state=0)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)
plot_data(X, y)

In [ ]:
model = Sequential(
    Linear(2,4),
    Tanh(),
    Linear(4,2),
    Tanh(),
    Linear(2,1),
    Sigmoid()
)
criterion = BCELoss()
optimizer = Adam(model.parameters(), lr=0.1)
model, history = trainer(model, X_tensor, y_tensor, optimizer, criterion, 100)
plot_loss_accuracy(history)

In [ ]:
plot_decision_boundary(model_forward_func(model), X, y)

In [ ]:
X, y = make_sine_wave()
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)
plot_data(X, y)

In [ ]:
X, y = make_multiclass(K=3)
y_cat = to_categorical(y)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_cat, dtype=torch.float32)


In [ ]:
from torch
model = Sequential(Linear(2, 3))
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.1)
model, history = trainer(model, X_tensor, y_tensor, optimizer, criterion, 100)
plot_loss_accuracy(history)

### IRIS dataset

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
import matplotlib.pyplot as plt

_, ax = plt.subplots()
scatter = ax.scatter(iris.data[:, 0], iris.data[:, 1], c=iris.target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[1])
_ = ax.legend(
    scatter.legend_elements()[0], iris.target_names, loc="lower right", title="Classes"
)

In [ ]:
# unused but required import for doing 3d projections with matplotlib < 3.2


from sklearn.decomposition import PCA

fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d", elev=-150, azim=110)

X_reduced = PCA(n_components=3).fit_transform(iris.data)
ax.scatter(
    X_reduced[:, 0],
    X_reduced[:, 1],
    X_reduced[:, 2],
    c=iris.target,
    s=40,
)

ax.set_title("First three PCA dimensions")
ax.set_xlabel("1st Eigenvector")
ax.xaxis.set_ticklabels([])
ax.set_ylabel("2nd Eigenvector")
ax.yaxis.set_ticklabels([])
ax.set_zlabel("3rd Eigenvector")
ax.zaxis.set_ticklabels([])

plt.show()

In [ ]:
history = History()
history.history['loss'] = []
history.history['acc'] = []
for e in range(50):
    
    y_pred_tensor = torch.squeeze(model(X_tensor))
    loss = criterion(y_pred_tensor, y_tensor)
    y_pred = y_pred_tensor.round().detach().numpy()
    
    if e % 10 == 0:
        print('Epoch:', e, 'Loss:', loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total = 0
    correct = 0
    total += y_tensor.size(0)
    correct += np.sum(y_pred == y)
    accuracy = correct/total

    history.history['loss'].append(loss.item())
    history.history['acc'].append(accuracy)

    history.epoch.append(e)